In [ ]:
import wave
import sys
import matplotlib.pyplot as plt
from scipy.io.wavfile import read
import librosa.display
import numpy as np
from scipy import stats
from spafe.frequencies.fundamental_frequencies import FundamentalFrequenciesExtractor
from spafe.frequencies.dominant_frequencies import get_dominant_frequencies 
from scipy.fft import fft,fftfreq,fftshift
import librosa
import os
import pandas as pd
from tqdm import tqdm
import re

In [ ]:
def spectral_properties(y: np.ndarray, fs: int) -> dict:
    """ This function takes signal and its rate,
    Outputs the feature values which we are going to explain here"""
    spec = np.abs(np.fft.rfft(y)) # Uses one dimensional Fast Furie Transformation and finds the absolute value of it 
    freq = np.fft.rfftfreq(len(y), d=1 / fs) # Discrete fast furie transformation
    amp = spec / spec.sum()
    mean = (freq * amp).sum()
    sd = np.sqrt(np.sum(amp * ((freq - mean) ** 2)))
    amp_cumsum = np.cumsum(amp)
    median = freq[len(amp_cumsum[amp_cumsum <= 0.5]) + 1]
    mode_val = freq[amp.argmax()]
    
    Q25 = freq[len(amp_cumsum[amp_cumsum <= 0.25]) + 1]
    Q75 = freq[len(amp_cumsum[amp_cumsum <= 0.75]) + 1]
    IQR = Q75 - Q25
    z = amp - amp.mean()
    w = amp.std()
    skew = ((z ** 3).sum() / (len(spec) - 1)) / w ** 3
    kurt = ((z ** 4).sum() / (len(spec) - 1)) / w ** 4
    sp_entr = -np.sum(amp*  np.log(amp))/np.log(len(amp))
    
    spectral_flatness = np.mean(librosa.feature.spectral_flatness(y,))
    
    fft_mode = np.mean(stats.mode(freq).mode)
    normalized_frequencies = np.linspace(0, 1, len(spec))
    spectral_centroid = np.sum(amp * normalized_frequencies)
    
    peakf = max(freq)
    
    fund_freqs_extractor = FundamentalFrequenciesExtractor(debug = False)
    pitches, harmonic_rates, argmins, times = fund_freqs_extractor.main(sig = y,fs = fs)
    meanf = np.mean(pitches)
    minf = min(pitches)
    maxf = max(pitches)
    
    
    dom_freq = get_dominant_frequencies(sig = np.array(y).reshape(-1,1),fs = int(fs),
                                        lower_cutoff = 50,
                                        upper_cutoff = 30000,
                                        nfft = 512,
                                        win_len = 0.02,
                                        win_hop = 0.015,
                                        win_type = 'hamming',
                                        debug = False)
    diff = np.abs(dom_freq[:-1] - dom_freq[1:])
    min_max = dom_freq.max() - dom_freq.min()
    if min_max == 0:
        mod_id = 0
    else:
        mod_id = diff.mean()/min_max
    
    dom_mean = dom_freq.mean()
    dom_min = dom_freq.min()
    dom_max = dom_freq.max()
    dom_max_min = dom_max - dom_min
    
    
    result_d = {
        'mean': mean,
        'sd': sd,
        'median': median,
        'mode_val': mode_val,
        'Q25': Q25/fs,
        'Q75': Q75,
        'IQR': IQR,
        'skew': skew,
        'kurt': kurt,
        'sp.ent':sp_entr,
        'sfm' : spectral_flatness,
        'fft_mode' : fft_mode,
        'centroid' : spectral_centroid,
        'peakf' : peakf,
        'meanf' : meanf,
        'minf' : minf,
        'maxf' : maxf,
        'modid' : mod_id,
        'dom_mean' : dom_mean,
        'dom_max' : dom_max,
        'dom_min' : dom_min,
        'dom_max_min' : dom_max_min
    }

    return result_d

In [ ]:
df_info = pd.read_csv('../CREMA-D/SentenceFilenames.csv')
df_demo = pd.read_csv('../CREMA-D/VideoDemographics.csv')

In [ ]:
def find_sex(id : np.int64):
    return df_demo[df_demo["ActorID"]==id]["Sex"]

In [ ]:
list_of_voices = []

data_folder = '../CREMA-D/AudioWAV/'

for each in tqdm(os.listdir(data_folder)):

    list_of_voices.append(librosa.load(data_folder + each))



In [ ]:
import warnings
warnings.filterwarnings('ignore')

data_folder = '../CREMA-D/AudioWAV/'
list_of_data = []
names = []
labels = []
for each in tqdm(os.listdir(data_folder)):

    sample, sample_rate = librosa.load(data_folder + each)
    smp = spectral_properties(sample,sample_rate)
    names = list(smp.keys()) + ["label"]
    lb = find_sex(int(re.findall(r'\d+', each)[0])).values[0]
    labels.append(lb)
    list_of_data.append(list(smp.values()) + [lb])



In [ ]:
df = pd.DataFrame(data = list_of_data,columns=names)

In [ ]:
df.to_csv("ExtractedFeatures.csv")

In [ ]:
df = pd.read_csv("ExtractedFeatures.csv")

In [ ]:
df.fillna(1e9,inplace = True)

In [ ]:
X = df.drop(["label"],axis = 1)

In [ ]:
y = df["label"]

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
model = RandomForestClassifier()

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y)

In [ ]:
model.fit(X_train, y_train)

In [ ]:
model.score(X_test,y_test)